In [1]:
pip install pandas mlxtend


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Step 1: Load the data
file_path = "C:/Users/josue/Desktop/WGU/D212/PA 3/medical_market_basket.csv"
data = pd.read_csv(file_path)
print("Data loaded successfully.")
print(data.head())







Data loaded successfully.
      Presc01            Presc02                    Presc03       Presc04  \
0         NaN                NaN                        NaN           NaN   
1  amlodipine  albuterol aerosol                allopurinol  pantoprazole   
2         NaN                NaN                        NaN           NaN   
3  citalopram            benicar  amphetamine salt combo xr           NaN   
4         NaN                NaN                        NaN           NaN   

     Presc05     Presc06     Presc07      Presc08     Presc09      Presc10  \
0        NaN         NaN         NaN          NaN         NaN          NaN   
1  lorazepam  omeprazole  mometasone  fluconozole  gabapentin  pravastatin   
2        NaN         NaN         NaN          NaN         NaN          NaN   
3        NaN         NaN         NaN          NaN         NaN          NaN   
4        NaN         NaN         NaN          NaN         NaN          NaN   

  Presc11   Presc12                  Presc

In [3]:
# Step 2: Check for missing values
missing_values = data.isna().sum()
print("Missing values in each column:")
print(missing_values)


Missing values in each column:
Presc01     7501
Presc02     9255
Presc03    10613
Presc04    11657
Presc05    12473
Presc06    13138
Presc07    13633
Presc08    14021
Presc09    14348
Presc10    14607
Presc11    14746
Presc12    14848
Presc13    14915
Presc14    14955
Presc15    14977
Presc16    14994
Presc17    14998
Presc18    14998
Presc19    14999
Presc20    15001
dtype: int64


In [4]:
# For some reason, the provided dataset has every line of data separated by a blank line. We don't want these rows
data = data[data['Presc01'].notna()]
# Reset index while we're at it, so we're not "missing" every other row
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Presc01  7501 non-null   object
 1   Presc02  5747 non-null   object
 2   Presc03  4389 non-null   object
 3   Presc04  3345 non-null   object
 4   Presc05  2529 non-null   object
 5   Presc06  1864 non-null   object
 6   Presc07  1369 non-null   object
 7   Presc08  981 non-null    object
 8   Presc09  654 non-null    object
 9   Presc10  395 non-null    object
 10  Presc11  256 non-null    object
 11  Presc12  154 non-null    object
 12  Presc13  87 non-null     object
 13  Presc14  47 non-null     object
 14  Presc15  25 non-null     object
 15  Presc16  8 non-null      object
 16  Presc17  4 non-null      object
 17  Presc18  4 non-null      object
 18  Presc19  3 non-null      object
 19  Presc20  1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


In [5]:
# Verify that NaN rows are gone
data.head()

,Presc01,Presc02,Presc03,Presc04,Presc05,Presc06,Presc07,Presc08,Presc09,Presc10,Presc11,Presc12,Presc13,Presc14,Presc15,Presc16,Presc17,Presc18,Presc19,Presc20
0,amlodipine,albuterol aerosol,allopurinol,pantoprazole,lorazepam,omeprazole,mometasone,fluconozole,gabapentin,pravastatin,cialis,losartan,metoprolol succinate XL,sulfamethoxazole,abilify,spironolactone,albuterol HFA,levofloxacin,promethazine,glipizide
1,citalopram,benicar,amphetamine salt combo xr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,enalapril,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,paroxetine,allopurinol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abilify,atorvastatin,folic acid,naproxen,losartan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Store data in a big list of lists
temp_big_list = []
# Iterate through each row, and within each row, iterate through each column
for row_number in range(len(data)):
    # Generate a temporary small list for each row
    temp_small_list = []
    for cell in range(len(data.columns)):
        # Check that cell contents are NOT null (we don't want to carry forth NaNs into the resulting Dataframe)
        if not pd.isnull(data.iloc[row_number, cell]):
            # If cell contents are not null (so, a prescription is present) then add a string version of that cell's contents
            # to the temporary small list
            temp_small_list.append(str(data.values[row_number, cell]))
    # Add the small list to the ongoing big list, for our list of lists
    temp_big_list.append(temp_small_list)
# Check that temp_big_list looks how we expect (a list of lists) by checking a few entries
print(f"Checking list of lists... \nindex 0: {temp_big_list[0]}\nindex 1: {temp_big_list[1]}\n...\nindex7500: {temp_big_list[7500]}")

Checking list of lists... 
index 0: ['amlodipine', 'albuterol aerosol', 'allopurinol', 'pantoprazole', 'lorazepam', 'omeprazole', 'mometasone', 'fluconozole', 'gabapentin', 'pravastatin', 'cialis', 'losartan', 'metoprolol succinate XL', 'sulfamethoxazole', 'abilify', 'spironolactone', 'albuterol HFA', 'levofloxacin', 'promethazine', 'glipizide']
index 1: ['citalopram', 'benicar', 'amphetamine salt combo xr']
...
index7500: ['amphetamine salt combo xr', 'levofloxacin', 'diclofenac sodium', 'cialis']


In [7]:
# Instantiate the transaction encoder
encoder = TransactionEncoder()
# Fit the transaction encoder to our list of lists, and then transform that data and store it in a temporary array
temp_array = encoder.fit(temp_big_list).transform(temp_big_list)
# Generate a new dataframe from this temporary array
clean_data = pd.DataFrame(temp_array, columns=encoder.columns_)
# Check that the new dataframe looks how we'd expect
clean_data

,Duloxetine,Premarin,Yaz,abilify,acetaminophen,actonel,albuterol HFA,albuterol aerosol,alendronate,allopurinol,...,trazodone HCI,triamcinolone Ace topical,triamterene,trimethoprim DS,valaciclovir,valsartan,venlafaxine XR,verapamil SR,viagra,zolpidem
0,False,False,False,True,False,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
clean_data.info()
# Save dataframe to CSV, ignore index (if included, this will create an additional unnecessary column)
clean_data.to_csv('task3_full_clean.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Columns: 119 entries, Duloxetine to zolpidem
dtypes: bool(119)
memory usage: 871.8 KB


In [9]:
# Use the Apriori algorithm to generate frequent itemsets
frequent_itemsets = apriori(clean_data, min_support = 0.02, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.046794,(Premarin)
1,0.238368,(abilify)
2,0.020397,(albuterol aerosol)
3,0.033329,(allopurinol)
4,0.079323,(alprazolam)
...,...,...
98,0.023064,"(lisinopril, diazepam)"
99,0.023464,"(losartan, diazepam)"
100,0.022930,"(metoprolol, diazepam)"
101,0.020131,"(glyburide, doxycycline hyclate)"


In [10]:
# Use association_rules with a lift of greater than 1 (antecedent increasing likelihood of consequent)
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 1.0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(amlodipine),(abilify),0.071457,0.238368,0.023597,0.330224,1.385352,0.006564,1.137144,0.299568
1,(abilify),(amlodipine),0.238368,0.071457,0.023597,0.098993,1.385352,0.006564,1.030562,0.365218
2,(amphetamine salt combo),(abilify),0.068391,0.238368,0.024397,0.356725,1.496530,0.008095,1.183991,0.356144
3,(abilify),(amphetamine salt combo),0.238368,0.068391,0.024397,0.102349,1.496530,0.008095,1.037830,0.435627
4,(abilify),(amphetamine salt combo xr),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562
...,...,...,...,...,...,...,...,...,...,...
89,(diazepam),(metoprolol),0.163845,0.095321,0.022930,0.139951,1.468215,0.007312,1.051893,0.381390
90,(glyburide),(doxycycline hyclate),0.170911,0.095054,0.020131,0.117785,1.239135,0.003885,1.025766,0.232768
91,(doxycycline hyclate),(glyburide),0.095054,0.170911,0.020131,0.211781,1.239135,0.003885,1.051852,0.213256
92,(losartan),(glyburide),0.132116,0.170911,0.028530,0.215943,1.263488,0.005950,1.057436,0.240286


In [11]:
top_3_rules = rules[(rules['lift'] > 1.9) & (rules['confidence'] > 0.3)].sort_values(by=['lift'], ascending= False)
top_3_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
75,(lisinopril),(carvedilol),0.098254,0.174110,0.039195,0.398915,2.291162,0.022088,1.373997,0.624943
72,(glipizide),(carvedilol),0.065858,0.174110,0.022930,0.348178,1.999758,0.011464,1.267048,0.535186
30,(metformin),(abilify),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255,0.503221


In [12]:
rules.antecedents.value_counts()

antecedents
(abilify)                      18
(carvedilol)                   12
(amphetamine salt combo xr)     9
(diazepam)                      8
(atorvastatin)                  7
(glyburide)                     6
(metoprolol)                    5
(lisinopril)                    4
(doxycycline hyclate)           4
(losartan)                      4
(citalopram)                    4
(glipizide)                     2
(amphetamine salt combo)        2
(amlodipine)                    2
(dextroamphetamine XR)          1
(clopidogrel)                   1
(fenofibrate)                   1
(levofloxacin)                  1
(metformin)                     1
(cialis)                        1
(naproxen)                      1
Name: count, dtype: int64

In [13]:
rules.consequents.value_counts()

consequents
(abilify)                      18
(carvedilol)                   12
(amphetamine salt combo xr)     9
(diazepam)                      8
(atorvastatin)                  7
(glyburide)                     6
(metoprolol)                    5
(doxycycline hyclate)           4
(lisinopril)                    4
(losartan)                      4
(citalopram)                    4
(amlodipine)                    2
(amphetamine salt combo)        2
(glipizide)                     2
(dextroamphetamine XR)          1
(levofloxacin)                  1
(clopidogrel)                   1
(metformin)                     1
(naproxen)                      1
(cialis)                        1
(fenofibrate)                   1
Name: count, dtype: int64

In [14]:
ant_data = rules[rules['antecedents'] == {'cialis'}]
con_data = rules[rules['consequents'] == {'cialis'}]
cialis_data = pd.concat([ant_data, con_data])
cialis_data

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
10,(cialis),(abilify),0.076523,0.238368,0.023997,0.313589,1.315565,0.005756,1.109585,0.259747
11,(abilify),(cialis),0.238368,0.076523,0.023997,0.100671,1.315565,0.005756,1.026851,0.314943


In [15]:
# Step 7: Save the cleaned data
cleaned_file_path = "C:/Users/josue/Desktop/WGU/D212/PA 3/cleaned_medical_market_basket.csv"
data.to_csv(cleaned_file_path, index=False)
print(f"Cleaned data saved to '{cleaned_file_path}'.")

Cleaned data saved to 'C:/Users/josue/Desktop/WGU/D212/PA 3/cleaned_medical_market_basket.csv'.
